In [7]:
import random
from importlib import reload

import torch

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from util import mt

In [8]:
model_name = 'EleutherAI/gpt-neo-125M'
model_name = 'facebook/xglm-7.5B'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).eval().half().cuda()

In [16]:
src = load_dataset("gsarti/flores_101", 'fin')
tgt = load_dataset("gsarti/flores_101", 'eng')

/gscratch/zlab/ahai/miniconda3/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/fin/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

/gscratch/zlab/ahai/miniconda3/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
src_endonym = 'suomi'
tgt_endonym = 'english'

In [18]:
def s_ids(s):
  return tokenizer(s).input_ids[1:]

In [19]:
def make_demo_prompt(src_endonym, tgt_endonym, src_points, tgt_points):
  demos = []
  for s, t in zip(src_points, tgt_points):
    demo = f"{src_endonym}: {s['sentence']} | {tgt_endonym}: {t['sentence']}"
    demos.append(demo)
  demo_prompt_ids = [2]
  for demo in demos:
    demo_prompt_ids.extend(s_ids(demo))
    demo_prompt_ids.append(2)
  return demo_prompt_ids

In [20]:
def make_unconditional_prompt(tgt, demo_tgts):
  demo_prompt_ids = [2]
  for t in demo_tgts:
    demo_ids = s_ids(f"{t['sentence']}")
    demo_prompt_ids.append(demo)
    demo_prompt_ids.append([2])
  return demo_prompt_ids

In [21]:
k=32
src_points = random.sample(list(src['devtest']), k=k)
tgt_points = [ tgt['devtest'][point['id']-1] for point in src_points ]

In [22]:
demo_prompt_ids = make_demo_prompt(src_endonym, tgt_endonym, src_points, tgt_points)

In [23]:
inference_point = src['dev'][1] 
prompt_ids = demo_prompt_ids + s_ids(f"{src_endonym}: {inference_point['sentence']} | {tgt_endonym}:")
prompt_t = torch.LongTensor([prompt_ids]).to(model.device)

In [24]:
print(tokenizer.decode(prompt_ids))

</s> suomi: Infrapunakuvien perusteella vuorokauden lämpötilavaihtelut vuorokauden osoittavat niiden olevan todennäköisesti luolia. | english: Infrared images show that the temperature variations from night and day show that they are likely caves.</s> suomi: Muista myös ääntää r ja rr eri tavalla. Caro tarkoittaa rakasta, carro vaunuja. | english: Also, make sure to pronounce r and rr differently: caro means dear, whereas carro means chariot.</s> suomi: Ratapölkyt otettiin käyttöön melko aikaisin pitämään raiteet paikoillaan. Vähitellen kuitenkin ymmärrettiin, että raiteet olisivat tehokkaampia, jos niiden päällä olisi rautainen kaistale. | english: Crossties were introduced fairly early to hold the tracks in place. Gradually, however, it was realised that tracks would be more efficient if they had a stip of iron on the top.</s> suomi: Vatikaanivaltio käyttää italian kieltä lainsäädännössään ja virallisessa viestinnässään. | english: Vatican City uses Italian in its legislation and off

In [30]:
with torch.no_grad():
  gen_ids = mt.greedy(model, prompt_t, 100)

In [31]:
tokenizer.decode(gen_ids[0].tolist())

'Leading researchers say that by using it, cancer, tuberculosis, HIV and malaria can be detected in early stages of patients living in low-income countries, where for example, breast cancer survival rate can be half that of richer countries.</s> suomi: Hän on ollut mukana monissa hankkeissa, jotka ovat edistäneet tieteen ja teknologian kehitystä. | english: He has been involved in many projects that have promoted science and technology.</s> suomi:'

In [29]:
tgt['dev'][1]

{'id': 2,
 'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'topic': 'health',
 'has_image': 0,
 'has_hyperlink': 0,
 'sentence': 'Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.'}

In [ ]:
tokenizer.decode([2] + gen_ids[0].tolist())

In [52]:
prompt_ids = torch.LongTensor([tokenizer('translate into english, 中文： 谢谢你！').input_ids[:-1]]).to(model.device)

In [39]:
prompt_ids = torch.LongTensor([tokenizer('中文： 谢谢你!').input_ids + [2] + tokenizer('english:').input_ids[1:]]).to(model.device)
                               
                               

In [53]:
prompt_ids.tolist()

[[2, 178680, 1531, 83763, 4, 6, 19773, 13, 6, 13305, 13305, 710]]

In [60]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_ids, 200)

In [61]:
tokenizer.decode([2] + gen_ids[0].tolist())

'</s> (English, Chinese)</s> 据说,懂得‘谢谢’这个字的人,都有大智慧。那是因为懂得‘谢谢’的人有聪明的心,即是知足常乐的智慧。那么就会明白每个人来到人间的时候,经过十二玄门的艰难关卡,习得一种深刻的智识,这智识就是:舍得福。</s> 舍得,用舍得的智慧,用也、不也、有、无(4种开关)设,把世间的种地、食材、人、<unk>!生命所需,拿 as;拿 came;拿 take;拿 go、拿 do、拿 make;拿 lie和 etc.加..所以懂得舍得的智慧人,他的心胸开阔,能够洞悉行人的对工夫、思想、心情!用精练的心、深情的眼睛、严肃的态度、轻快的语气、亲切的笑'

In [51]:
print(tokenizer.decode(prompt_ids[0]))

</s> 中文: 谢谢你! | english


In [14]:
inference_point['sentence']

'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'

In [22]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [12]:
vocab = tokenizer.get_vocab()
tokens = list(sorted(tokenizer.get_vocab(), key=lambda k: vocab[k]))

In [13]:
tokens[:10]

['<s>', '<pad>', '</s>', '<unk>', ',', '.', '▁', 's', '-', 'a']

In [31]:
model.model.embed_tokens.weight

Parameter containing:
tensor([[-0.0079,  0.0080, -0.0081,  ..., -0.0215,  0.0009, -0.0312],
        [-0.0011, -0.0034,  0.0039,  ..., -0.0026,  0.0019, -0.0105],
        [ 0.0152,  0.1249, -0.0972,  ..., -0.0403, -0.0509,  0.0708],
        ...,
        [-0.0106, -0.0220, -0.0239,  ..., -0.0101,  0.0071, -0.0078],
        [-0.0041, -0.0020, -0.0120,  ..., -0.0260,  0.0097, -0.0233],
        [ 0.0101,  0.0316,  0.0064,  ..., -0.0074, -0.0031, -0.0039]],
       device='cuda:0', dtype=torch.float16, requires_grad=True)

In [33]:
with open('eng_val.txt', 'w') as out:
  for datum in tgt['dev']:
    out.write(f'{datum["sentence"]}\n')

In [37]:
src['dev'][0]

{'id': 1,
 'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'topic': 'health',
 'has_image': 0,
 'has_hyperlink': 0,
 'sentence': 'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'}

In [38]:
tgt['dev'][0]

{'id': 1,
 'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'topic': 'health',
 'has_image': 0,
 'has_hyperlink': 0,
 'sentence': 'On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.'}